In [44]:
import pandas as pd
df = pd.read_csv("The_Office_lines.csv")

In [68]:
# import first pre-trained sentiment analysis pipeline
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [45]:
relevant_columns = ["speaker", "line_text"]
df = df[relevant_columns]

In [59]:
import re

# deals with descriptions in lines, e.g. [laughs] or [to camera]
def deal_with_description(line, mode):
    if mode=="remove":
        # remove text that is between brackets
        line = re.sub(r'\[.*?\]', '', line)
    elif mode=="end":
        # move all the text that is in the brackets to the end of the line
        line = re.sub(r'\[.*?\]', '', line) + " " + ", ".join(re.findall(r"\[(.*?)\]", line))
    elif mode=="start":
        # move all the text that is in the brackets to the start of the line
        line = ", ".join(re.findall(r"\[(.*?)\]", line)) + " " + re.sub(r'\[.*?\]', '', line)
    elif mode=="keep":
        # remove all brackets from the line but keep text in place
        line = re.sub(r"[\([{})\]]", '', line)
    return line

def preprocess_sentiment(df, relevant_columns, description_mode):
    # filter out relevant columns
    df = df[relevant_columns]
    # deal with descriptions in lines
    df["line_text"] = df["line_text"].apply(lambda x: deal_with_description(x, mode=description_mode))
    # tokenize lines
    
    return df

In [65]:
df_filtered = preprocess_sentiment(df, relevant_columns, description_mode="keep")

In [69]:
# apply sentiment analysis to lines
df_filtered["sentiment"] = df_filtered["line_text"].apply(lambda x: sentiment_analysis(x)[0]["label"])

KeyboardInterrupt: 